In [3]:
include("../src/JuTrack.jl")

Main.JuTrack

In [4]:
using .JuTrack
using Enzyme
using Test
using BenchmarkTools

In [5]:
Threads.nthreads()

8

In [2]:
function fodo_ring(K1, RAD)
        D1 = DRIFT(name="D1", len=0.34)
        D2 = DRIFT(name="D2", len=0.59)
        QF1 = KQUAD(name="QF1", len=0.32, k1=K1, rad=RAD )
        QD1 = KQUAD(name="QD1", len=0.32, k1=-1.192160, rad=RAD  )
        BendingAngle = pi/2
        BD1 = SBEND(name="BD1", len=0.72, angle=BendingAngle/2, e1=BendingAngle/2, e2=0.0 , rad=RAD )
        BD2 = SBEND(name="BD2", len=0.72, angle=BendingAngle/2, e1=0.0, e2=BendingAngle/2, rad=RAD  )

        FODO = (QF1, D1, QD1, D1, QF1)
        BD1_comp = (BD1)
        BD2_comp = (BD2)
        D2_comp = (D2)

        CELL = Vector{AbstractElement}(undef, 8)
        for i in eachindex(FODO)
                CELL[i] = FODO[i]
        end
        CELL[length(FODO)+1] = BD1
        CELL[length(FODO)+2] = D2
        CELL[length(FODO)+3] = BD2
                # CELL[i+172] = M5[i]
        
        ELIST = Vector{AbstractElement}(undef, 4*length(CELL))
        for i in eachindex(CELL)
                ELIST[i] = CELL[i]
                ELIST[i+length(CELL)] = CELL[i]
                ELIST[i+2*length(CELL)] = CELL[i]
                ELIST[i+3*length(CELL)] = CELL[i]
        end
        return ELIST
end

function create_sbend(BendingAngle)
        SBD = SBEND(name="BD", len=0.72, angle=BendingAngle/2, e1=BendingAngle/2, e2=0.0 , rad=0.0)
        return [SBD]
end        

function sbend_track_mthread(BendingAngle)
        particles = [.001 .0001 .0005 .0002 0.0 0.0] 
        beam = Beam(particles)
        line = create_sbend(BendingAngle)
        plinepass!(line, beam)
        return beam.r
end

function sbend_track(BendingAngle)
        particles = [.001 .0001 .0005 .0002 0.0 0.0] 
        beam = Beam(particles)
        line = create_sbend(BendingAngle)
        linepass!(line, beam)
        return beam.r
end


sbend_track (generic function with 1 method)

One particle Tests:

In [109]:
# In bend.jl I changed the BendSymplecticPass_P! functions to reflect the multithreading by commenting out the for loop and adding the @threads macro.

bend_angle = pi/2
if sbend_track(bend_angle) == sbend_track_mthread(bend_angle)
        println("Multithreading matches single thread tracking")
else
        println("Multithreading DOES NOT match single thread tracking")
end


grad1 = autodiff(Forward, sbend_track, DuplicatedNoNeed, Duplicated(bend_angle, 1.0))
grad2 = autodiff(Forward, sbend_track_mthread, DuplicatedNoNeed, Duplicated(bend_angle, 1.0))

if grad1 == grad2
        println("Multithreading matches single thread AD")
else
        println("Multithreading DOES NOT match single thread AD")
end
println("")
println("")
# Benchmarking


time_track = @timed sbend_track(bend_angle)
time_track_mthread = @timed sbend_track_mthread(bend_angle)
println("Single Thread Time: ", time_track[2])
println("Multi Thread Time: ", time_track_mthread[2])
if time_track[2] < time_track_mthread[2]
        println("Single thread is faster by " * string(time_track_mthread[2] - time_track[2]) * " seconds")
else
        println("Multithread is faster by " * string(time_track[2] - time_track_mthread[2]) * " seconds")
end

println("It seems that the multithread has a bit of variablity in the time it takes to run. Sometimes it is on the same order as single thread, sometimes it is 2-3 orders of magnitude slower")

println("Single Thread Memory: ", time_track[3] , " bytes")
println("Multi Thread Memory: ", time_track_mthread[3], " bytes")
if time_track[2] < time_track_mthread[2]
        println("Single thread is faster by " * string(time_track_mthread[2] - time_track[2]) * " seconds")
else
        println("Multithread is faster by " * string(time_track[2] - time_track_mthread[2]) * " seconds")
end

println(time_track_mthread)


# println("time_sbend_track: ", time_track[3])
# println("")
# println("")
# println("Multi Thread")
# @timed sbend_track_mthread(bend_angle)

# @timev sbend_track(bend_angle)
# @timev sbend_track_mthread(bend_angle)
# println(time_track[2], " ", time_track_mthread[2], " ", time_track[2] - time_track_mthread[2])
# println(time_track[2], " ", time_track_mthread[2])





Multithreading matches single thread tracking
Multithreading matches single thread AD


Single Thread Time: 3.01e-5
Multi Thread Time: 0.0023
Single thread is faster by 0.0022699 seconds
It seems that the multithread has a bit of variablity in the time it takes to run. Sometimes it is on the same order as single thread, sometimes it is 2-3 orders of magnitude slower
Single Thread Memory: 4560 bytes
Multi Thread Memory: 11728 bytes
Single thread is faster by 0.0022699 seconds
(value = [0.0014790373312097663 7.071499198396235e-5 0.0002513009183012763 -0.0003454153912482279 0.0010270873405908145 0.0], time = 0.0023, bytes = 11728, gctime = 0.0, gcstats = Base.GC_Diff(11728, 0, 0, 88, 0, 0, 0, 0, 0))


In [97]:
# println(@benchmark sbend_track(bend_angle))
# @benchmark sbend_track_mthread(bend_angle)

Trial(3.125 μs)


BenchmarkTools.Trial: 5142 samples with 1 evaluation.
 Range (min … max):    5.800 μs … 31.680 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      15.900 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   967.205 μs ±  1.190 ms  ┊ GC (mean ± σ):  0.06% ± 0.88%

  █                          ▂▄▅▃▄▂▃▂▁▁▁▁▁▁▁▁                  ▁
  █▅▃▄▄▁▁▃▁▃▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▃▆██████████████████████▇▇▇▇▇▆▅▄▅▆▆ █
  5.8 μs        Histogram: log(frequency) by time      3.39 ms <

 Memory estimate: 11.20 KiB, allocs estimate: 80.

In [106]:
@time sbend_track(bend_angle)

  0.000034 seconds (36 allocations: 4.453 KiB)


1×6 Matrix{Float64}:
 0.00147904  7.0715e-5  0.000251301  -0.000345415  0.00102709  0.0